Storing and Resampling Real Time Data (Part2)

In [1]:
import pandas as pd
import tpqoa
import time

In [10]:
class GetTickData(tpqoa.tpqoa):
    def __init__(self, config_file, bar_length):
        super().__init__(config_file)
        self.bar_length = bar_length
        self.tick_data = pd.DataFrame()
        self.tick_count = 0
        self.target_ticks = None
        self.stop = False

    def on_success(self, time, bid, ask):
        print(f"Tick {self.tick_count + 1}/20: {time}, Bid: {bid}, Ask: {ask}")
        df = pd.DataFrame({
            "bid": bid, 
            "ask": ask, 
            "mid": (ask + bid)/2
        }, index=[pd.to_datetime(time)])
        
        self.tick_data = pd.concat([self.tick_data, df])
        self.data = self.tick_data.resample(self.bar_length, label="right").last().ffill()
        
        self.tick_count += 1
        if self.target_ticks and self.tick_count >= self.target_ticks:
            self.stop = True

    def stream_data(self, instrument, stop=None, ret=False, callback=None):
        self.tick_count = 0
        self.target_ticks = stop
        self.stop = False
        super().stream_data(instrument, stop=None, ret=ret, callback=callback)

In [12]:
td = GetTickData("oanda.cfg", "5s")

In [14]:
td.stream_data("EUR_USD", stop=20)

Tick 1/20: 2024-10-04T20:59:05.145319455Z, Bid: 1.09737, Ask: 1.09763


V20Timeout: v20 REST request to https://stream-fxpractice.oanda.com:443/v3/accounts/101-001-29655670-001/pricing/stream?instruments=EUR_USD&snapshot=True has timed out (stream)

In [ ]:
print("\nFinal resampled data:")
print(td.data)